### Hypothetical Document Embeddings(HyDE)
HyDE is a retrival technique where, instead of **embedding the user's query directly, we first generate a hypothetical answer(document) to the query usint llm** - and then embedded the hypothetical document to search your vector store.

##### Hyde bridges the gap b/w user intent and relevent content, especially when:
1. Queries are Short
2. Language mismatch between query and documents
3. You want to retrieve based on answer content, not question words.

In [22]:
from langchain_classic.document_loaders import WikipediaLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain.chat_models import init_chat_model
from langchain_classic.output_parsers import StructuredOutputParser
from langchain_classic.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate


In [4]:
# 1. Load and chunk your dataset
chunk_size = 300
chunk_overlap = 100

# loading data
loader = WikipediaLoader(query="Steve Jobs", load_max_docs=5)
documents = loader.load()

# text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
docs = text_splitter.split_documents(documents=documents)
# docs

In [ ]:
# 2. Build vector Store
embedding_model = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embedding_model)


In [16]:
# 3. Setup the LLM you'l use to generate hypothetical answers
from dotenv import load_dotenv
load_dotenv()

llm = init_chat_model("groq:groq/compound-mini")
llm

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x0000022535963770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022535AFC950>, model_name='groq/compound-mini', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [17]:
## Creating vector stores 
db = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory="output/steve_jobs")
# create the retriver
base_retriever = db.as_retriever(search_kwargs ={"k":5})

In [20]:
## Generating a prompt for generating HyDE
def get_hyde_doc(query):
    system_template = """Imagine you are an expert writing a detailed explanation."""
    
    human_template = """Write a detailed hypothetical answer for the topic: {query}"""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([
        system_message_prompt,
        human_message_prompt
    ])

    messages = chat_prompt.format_prompt(query=query).to_messages()
    
    response = llm.invoke(messages)
    return response.content

In [23]:
query = "When was Steve Jobs fired from apple?"
print(get_hyde_doc(query=query))

**When Was Steve Jobs Fired from Apple? – A Detailed Account**

*Short answer:*  
Steve Jobs was effectively forced out of Apple in **September 1985** after a board‑level power struggle with then‑CEO John Sculley and other senior executives. The formal resignation of his title as “Head of the Macintosh Division” was submitted on **September 17, 1985**, and by the end of that month he had left the company for good.

Below is a step‑by‑step narrative that explains how and why this happened, the key players involved, and what the aftermath looked like.

---

## 1. The Early Years (1976‑1983)

| Year | Event | Significance |
|------|-------|--------------|
| **1976** | Steve Jobs, Steve Wozniak, and Ronald Wayne co‑found Apple Computer, Inc. | Jobs becomes the charismatic, product‑obsessed visionary; Wozniak supplies the engineering brilliance. |
| **1977** | Apple II released | First major commercial success; Apple goes public in 1980. |
| **1983** | John Sculley, former PepsiCo marketing

In [26]:
matched_doc = base_retriever.invoke(get_hyde_doc(query))
print(matched_doc)

[Document(metadata={'title': 'Steve Jobs', 'summary': 'Steven Paul Jobs  (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers. \nJobs saw the commercial potential

#

# Langchain - HypotheticalDocumentEmbedder


In [27]:
from langchain_classic.chains.hyde.base import HypotheticalDocumentEmbedder

from langchain_classic.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_classic.document_loaders import TextLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.chains.combine_documents import create_stuff_documents_chain


In [29]:
# Step: 1 Load and split the documents
loader = TextLoader("langchain.txt", encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [31]:
# Step 2: Setup LLM and embedding
base_embedding = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
llm = init_chat_model("groq:groq/compound-mini")

According to the official documentation and Langchain source coe(mapping in PROMPT_MAP), the default options are:
- web_search
- sci_fact
- arguana
- trec_covid
- trec_news
- fiqa
etc..

In [40]:
# custom = PromptTemplate.from_template(
#     "Generate a concise hypothetical answer for this topic: {query}"
# )

# Step 3: HyDE Embedder using prompt_key='web_search'01_query_expansion.ipynb
hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embedding,
    prompt_key="web_search"
    # custom_prompt=custom
)

## Custom prompt


In [ ]:
custom = PromptTemplate.from_template(
    "Generate a concise hypothetical answer for this topic: {query}"
)

# Step 3: HyDE Embedder using custom_prompt
hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embedding,
    # custom_prompt=custom
)

##

In [33]:
# Step 4: Store documents in Chroma with HyDE embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=hyde_embedding_function,
    persist_directory="output/langchain"
)

In [37]:
# Step 5: RAG answer generation prompt
rag_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.
Context:{context}

Question: {input}                                                                      
""")
rag_chain = create_stuff_documents_chain(llm=llm,prompt=rag_prompt)

In [35]:
# Step 6: Final RAG Pipeline
def hyde_rag_pipeline(query):
    matched_docs = vector_store.similarity_search(query, k=4)
    response = rag_chain.invoke({
        "input": query,
        "context": matched_docs
    })
    return response

In [38]:
# Step 7: Run example query
query = "What memory modules does Langchain provide?"
answer = hyde_rag_pipeline(query)
print("☑️ Final Answer: \n ", answer)

☑️ Final Answer: 
  LangChain ships with a variety of built‑in memory classes that you can plug directly into a chain or agent. The most commonly used ones are:

| Memory class | What it does | Typical use‑case |
|--------------|--------------|-----------------|
| **ConversationBufferMemory** | Stores the full chat history as a plain text buffer. | Simple bots that need the entire conversation context. |
| **ConversationBufferWindowMemory** (or **BufferWindowMemory**) | Keeps only the last *k* turns. | Keeps context lightweight while still remembering recent dialogue. |
| **ConversationSummaryMemory** | Summarizes the whole conversation into a short text that is passed forward. | Long‑running chats where token cost would explode if you kept everything. |
| **ConversationSummaryBufferMemory** | Combination of a buffer and a periodic summary (keeps recent turns plus a running summary). | Mid‑size conversations where you want both recent detail and overall gist. |
| **EntityMemory** | Ext